In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
#from scipy.stats import norm
# from scipy import stats
# from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
non_coding_region="core_prom"
core_prom_type= "TATA"
cancer_type= "Brain"

In [3]:
base_dir = "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/DNABERT_Data/{}/Patient_wise".format(cancer_type, non_coding_region)
output_dir= "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/".format(cancer_type, core_prom_type)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
df_core_prom = pd.read_csv("/home/pdutta/Data/Noncoding_region/core_prom_bed.tsv", sep="\t")
df_core_prom

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr7,ENSG00000004059.11,ENST00000000233.10,ENSE00001872691.2,+,127588411,127588366,127588455
1,chr7,ENSG00000001626.16,ENST00000003084.11,ENSE00001343851.2,+,117480025,117479980,117480069
2,chr7,ENSG00000001630.17,ENST00000003100.13,ENSE00001216550.10,-,92134477,92134433,92134522
3,chr7,ENSG00000002933.9,ENST00000004103.8,ENSE00001827596.2,+,150800769,150800724,150800813
4,chr7,ENSG00000004799.8,ENST00000005178.6,ENSE00001133344.6,-,95596516,95596472,95596561
...,...,...,...,...,...,...,...,...
236970,chr15,ENSG00000103876.14,ENST00000684569.1,ENSE00003921718.1,+,80153010,80152965,80153054
236971,chr15,ENSG00000213614.11,ENST00000684602.1,ENSE00002620879.1,-,72375989,72375945,72376034
236972,chr15,ENSG00000213614.11,ENST00000684667.1,ENSE00002584031.1,-,72375972,72375928,72376017
236973,chr15,ENSG00000104133.16,ENST00000684676.1,ENSE00003922470.1,-,44663688,44663644,44663733


In [5]:
# Grouping by 'Chr', 'strand', 'TSS', 'start', and 'end'
grouped = df_core_prom.groupby(['Chr', 'strand', 'TSS', 'start', 'end'])

# Aggregating 'gene_id', 'transcript_id', and 'exon_id' to get unique lists
aggregated = grouped.agg({
    'gene_id': lambda x: list(x.unique()),
    'transcript_id': lambda x: list(x.unique()),
    'exon_id': lambda x: list(x.unique())
}).reset_index()

# Reordering the columns as specified
df_core_prom = aggregated[['Chr', 'start', 'end', 'strand', 'TSS', 'gene_id', 'transcript_id', 'exon_id']]

In [6]:
df_core_prom

,Chr,start,end,strand,TSS,gene_id,transcript_id,exon_id
0,chr1,11824,11913,+,11869,[ENSG00000223972.5],[ENST00000456328.2],[ENSE00002234944.1]
1,chr1,11965,12054,+,12010,[ENSG00000223972.5],[ENST00000450305.2],[ENSE00001948541.1]
2,chr1,29509,29598,+,29554,[ENSG00000243485.5],[ENST00000473358.1],[ENSE00001947070.1]
3,chr1,30222,30311,+,30267,[ENSG00000243485.5],[ENST00000469289.1],[ENSE00001841699.1]
4,chr1,30321,30410,+,30366,[ENSG00000284332.1],[ENST00000607096.1],[ENSE00003695741.1]
...,...,...,...,...,...,...,...,...
209272,chrY,57016052,57016141,-,57016096,[ENSG00000237801.6_PAR_Y],[ENST00000412936.6_PAR_Y],[ENSE00001527118.2]
209273,chrY,57165801,57165890,-,57165845,[ENSG00000228410.6_PAR_Y],[ENST00000456370.6_PAR_Y],[ENSE00001674980.1]
209274,chrY,57172725,57172814,-,57172769,[ENSG00000223484.7_PAR_Y],[ENST00000421233.6_PAR_Y],[ENSE00001611457.1]
209275,chrY,57203313,57203402,-,57203357,[ENSG00000185203.12_PAR_Y],[ENST00000399966.9_PAR_Y],[ENSE00001772131.3]


In [ ]:
chr16_3133660_3133749_3133739_3133740_T_A

In [23]:
df_core_prom[df_core_prom['start']==3133660]

,Chr,start,end,strand,TSS,gene_id,transcript_id,exon_id
75258,chr16,3133660,3133749,-,3133704,[ENSG00000263072.8],"[ENST00000576590.2, ENST00000653667.1, ENST000...",[ENSE00003865797.1]


In [7]:
# Filtering rows where the list in 'transcript_id' has more than one element
df_core_prom.to_csv("/home/pdutta/Data/Noncoding_region/unique_core_prom_bed.tsv", index=False)

In [8]:
with open("/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Intersected_Data/intersected_vcf_{}_data.pkl".format(non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [9]:
len(loaded_dictionary)

432

In [10]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [22]:
patient_dfs= []
patient_id = []
dir_remain = []  # List to keep track of directories where the file is missing

for dirpath, dirnames, filenames in os.walk(base_dir):
    #print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        #print(dir_name)
        #input()
        df_temp = loaded_dictionary[dir_name]
        #print(len(df_temp))
        df_temp = df_temp.drop_duplicates(subset=['chr_name', 'start', 'end', 'transcript_id','START_POS', 'END_POS', 'REF','ALT']).reset_index()
        try:
            concat_probab = np.load(dir_path+"/{}/Prediction_result/pred_results.npy".format(core_prom_type))
            #print(len(concat_probab))
            #print(len(df_temp))
            if(2*len(df_temp)==len(concat_probab)):
                #input()
                Reference_probab = concat_probab[::2]
                Alternative_probab = concat_probab[1::2]
                df_temp['Ref_probab']=Reference_probab
                df_temp['Alt_probab']=Alternative_probab
                df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
                df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
                #display(df_temp)
                #input()
                # Apply the filter conditions
                df_temp = df_temp[((df_temp['Ref_probab'] > 0.5) & (df_temp['Alt_probab'] <= 0.5)) | 
                          ((df_temp['Ref_probab'] <= 0.5) & (df_temp['Alt_probab'] > 0.5))]
                #display(df_temp)
                #input()
                patient_dfs.append(df_temp)
            else:
                print(dir_name)
                input()
        except FileNotFoundError:
            print(f"File not found in directory: {dir_path}")
            dir_remain.append(dir_name)
        #input()
    break

/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_raw_pindel
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_raw_pindel
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_raw_pindel
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/67c83385-8497-4c2d-932e-9a3b9bb90ec9_CaVEMan
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/447a08c8-3488-45d1-9262-23c42341296f_sanger_raw_pindel
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Pat

In [12]:
dir_remain

[]

In [29]:
patient_dfs[0]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
191,233,chr7,83373052,83373141,ENSG00000285398.1,ENST00000644734.1,ENSE00003821943.1,-,83373096,chr7,...,D,83373129,1,83373123,12,423.3280029296875,0.999992,0.148406,-0.851579,19.498234
1204,1504,chr1,150153438,150153527,ENSG00000023902.14,ENST00000607609.2,ENSE00003699801.2,+,150153483,chr1,...,I,150153504,10,150153492,304,2775.56005859375,0.999990,0.000213,-0.999766,28.738061
1476,1857,chr9,91159528,91159617,ENSG00000235641.5,ENST00000423719.3,ENSE00003702836.1,+,91159573,chr9,...,I,91159595,4,91159572,40,969.2319946289062,0.999987,0.000222,-0.999752,28.394499
2810,3624,chr10,106140219,106140308,ENSG00000229775.7,ENST00000449761.5,ENSE00001724579.1,+,106140264,chr10,...,D,106140226,2,106140223,84,1441.3599853515625,0.999980,0.003074,-0.996886,23.957500
2860,3686,chr10,79968168,79968257,ENSG00000273372.1,ENST00000608229.1,ENSE00003708259.1,+,79968213,chr10,...,I,79968235,1,79968232,380,3120.159912109375,0.000037,0.999782,0.999527,-26.869702
3147,4040,chr2,229923195,229923284,ENSG00000153827.14,ENST00000435716.5,ENSE00001725866.1,-,229923239,chr2,...,D,229923259,1,229923253,54,1099.27001953125,0.000101,0.999996,0.999891,-31.228957


In [33]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).reset_index(drop=True)

if 'index' in unique_patient_regions.columns:
    unique_patient_regions.drop(columns=['index'], inplace=True)
unique_patient_regions = unique_patient_regions.drop_duplicates(subset=['chr_name', 'transcript_id', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT']).reset_index(drop=True)
unique_patient_regions

,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,START_POS,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
0,chr7,83373052,83373141,ENSG00000285398.1,ENST00000644734.1,ENSE00003821943.1,-,83373096,chr7,83373122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,150153438,150153527,ENSG00000023902.14,ENST00000607609.2,ENSE00003699801.2,+,150153483,chr1,150153491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr9,91159528,91159617,ENSG00000235641.5,ENST00000423719.3,ENSE00003702836.1,+,91159573,chr9,91159571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr10,106140219,106140308,ENSG00000229775.7,ENST00000449761.5,ENSE00001724579.1,+,106140264,chr10,106140222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr10,79968168,79968257,ENSG00000273372.1,ENST00000608229.1,ENSE00003708259.1,+,79968213,chr10,79968231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,chr10,79968168,79968257,ENSG00000273372.1,ENST00000608229.1,ENSE00003708259.1,+,79968213,chr10,79968231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1809,chr2,229923195,229923284,ENSG00000153827.14,ENST00000435716.5,ENSE00001725866.1,-,229923239,chr2,229923252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1810,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,23897307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1811,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,23897307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
concatenated_df.columns

Index(['index', 'chr_name', 'start', 'end', 'gene_id', 'transcript_id',
       'exon_id', 'strand', 'TSS', 'CHROM', 'START_POS', 'END_POS', 'ID',
       'REF', 'ALT', 'QUAL', 'FILTER', 'FF017', 'LEN', 'OLD_VARIANT', 'PC',
       'RE', 'REP', 'RS', 'S1', 'S2', 'Ref_probab', 'Alt_probab',
       'ScoreChange', 'LogOddRatio', 'ASMD', 'ASRD', 'CLPM', 'DP', 'DS', 'GP',
       'MP', 'SG', 'SNP', 'SP', 'TG', 'TP'],
      dtype='object')

In [34]:
# Concatenate the patient dataframes
concatenated_df = pd.concat(patient_dfs)

# Group by specified columns and aggregate 'transcript_id' into a comma-separated string
unique_patient_regions = concatenated_df.groupby(['chr_name', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT']).agg({
    'transcript_id': lambda x: ', '.join(x.unique())  # Aggregates into a unique, comma-separated list
}).reset_index()
unique_patient_regions

,chr_name,start,end,START_POS,END_POS,REF,ALT,transcript_id
0,chr1,11029614,11029703,11029633,11029638,CTTTT,C,ENST00000607145.1
1,chr1,37782841,37782930,37782904,37782906,CT,C,ENST00000417484.1
2,chr1,37782841,37782930,37782914,37782916,TA,T,ENST00000417484.1
3,chr1,89589498,89589587,89589557,89589558,C,CT,ENST00000660444.1
4,chr1,150153438,150153527,150153491,150153492,A,AT,ENST00000607609.2
...,...,...,...,...,...,...,...,...
105,chrX,13722196,13722285,13722209,13722215,AAAAAA,CC,ENST00000380578.1
106,chrX,13722196,13722285,13722209,13722216,AAAAAAA,GC,ENST00000380578.1
107,chrX,13722196,13722285,13722210,13722211,A,C,ENST00000380578.1
108,chrX,56737533,56737622,56737591,56737592,A,AT,ENST00000421129.1


In [15]:
filtered_regions = unique_patient_regions.groupby(['chr_name', 'transcript_id', 'start', 'end']).filter(lambda x: len(x) > 1 and not all(x['START_POS'].eq(x['START_POS'].iloc[0]) & x['END_POS'].eq(x['END_POS'].iloc[0]) & x['REF'].eq(x['REF'].iloc[0]) & x['ALT'].eq(x['ALT'].iloc[0])))
filtered_regions

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
1204,1504,chr1,150153438,150153527,ENSG00000023902.14,ENST00000607609.2,ENSE00003699801.2,+,150153483,chr1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1476,1857,chr9,91159528,91159617,ENSG00000235641.5,ENST00000423719.3,ENSE00003702836.1,+,91159573,chr9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3147,4040,chr2,229923195,229923284,ENSG00000153827.14,ENST00000435716.5,ENSE00001725866.1,-,229923239,chr2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2566,3288,chrX,13722196,13722285,ENSG00000196459.15,ENST00000380578.1,ENSE00001485528.1,-,13722240,chrX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3634,4653,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3635,4654,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2616,2616,chrX,13722196,13722285,ENSG00000196459.15,ENST00000380578.1,ENSE00001485528.1,-,13722240,chrX,...,0.0,52.0,.,0.04200,0.95,AA/AAACC,False,0.0720,AA/AAAAC,0.88
55,76,chr7,121686319,121686408,ENSG00000227743.2,ENST00000432702.2,ENSE00003835825.1,-,121686363,chr7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3915,5006,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1113,1482,chr1,202734271,202734360,ENSG00000117139.18,ENST00000472822.6,ENSE00003836587.1,-,202734315,chr1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pd.set_option('display.max_columns', None)
filtered_regions[filtered_regions['start']==114277279]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
281,346,chr13,114277279,114277368,ENSG00000285672.1,ENST00000649577.1,ENSE00003837885.1,-,114277323,chr13,114277317,114277324,f06a2dca-9990-11ea-a543-2154a91dbdb5,AAAAAAC,A,240.0,FF010,True,6.0,.,D,114277330.0,1.0,114277318.0,8.0,336.0669860839844,0.000018,0.998504,0.996992,-25.163649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,467,chr13,114277279,114277368,ENSG00000285672.1,ENST00000649577.1,ENSE00003837885.1,-,114277323,chr13,114277306,114277313,5f06af7c-2476-11eb-8dfa-b9428c171e86,TCAAAAA,T,240.0,FF010,True,6.0,.,D,114277314.0,1.0,114277307.0,5.0,337.8370056152344,0.000018,0.998738,0.997460,-25.409543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
unique_patient_regions[unique_patient_regions['start']==3133660]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
63,63,chr16,3133660,3133749,ENSG00000263072.8,ENST00000576590.2,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36
64,64,chr16,3133660,3133749,ENSG00000263072.8,ENST00000653667.1,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36
65,65,chr16,3133660,3133749,ENSG00000263072.8,ENST00000654387.1,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36
67,67,chr16,3133660,3133749,ENSG00000263072.8,ENST00000659227.1,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36
69,69,chr16,3133660,3133749,ENSG00000263072.8,ENST00000668778.1,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36
70,70,chr16,3133660,3133749,ENSG00000263072.8,ENST00000668823.1,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36
71,71,chr16,3133660,3133749,ENSG00000263072.8,ENST00000670666.1,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36
72,72,chr16,3133660,3133749,ENSG00000263072.8,ENST00000671396.1,ENSE00003865797.1,-,3133704,chr16,3133739,3133740,e403597c-57a2-11ec-a9ac-b453e2dd4bb0,T,A,.,SR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999218,0.000516,-0.997921,21.239011,87.0,0.58,0.0,16.0,.,0.18,0.82,TT/AAATT,False,0.23,TT/AATTT,0.36


In [28]:
# Reset the option to its default (typically 20)
pd.reset_option('display.max_columns')

In [19]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df)
    for _, region in unique_patient_regions.iterrows():
        #print(region)
        #input()
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr_name'] == region.chr_name) & (df['start'] == region.start)& (df['end'] == region.end) & (df['START_POS'] == region.START_POS) & (df['END_POS'] == region.END_POS) & (df['REF'] == region.REF) & (df['ALT'] == region.ALT) , score_column]
            #print(score_series)
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'].append(score)

(6, 30)
(10, 30)
(1, 33)
(4, 30)
(0, 33)
(9, 30)
(8, 30)
(7, 30)
(0, 33)
(15, 30)
(6, 30)
(5, 30)
(0, 33)
(1, 33)
(0, 33)
(0, 33)
(1, 33)
(0, 33)
(8, 30)
(8, 30)
(0, 33)
(9, 30)
(8, 30)
(0, 33)
(0, 33)
(9, 30)
(0, 33)
(0, 33)
(19, 30)
(0, 33)
(0, 33)
(6, 30)
(0, 33)
(9, 30)
(8, 30)
(9, 30)
(9, 30)
(0, 33)
(5, 30)
(0, 33)
(0, 33)
(8, 30)
(8, 30)
(0, 33)
(0, 33)
(0, 33)
(8, 30)
(3, 33)
(0, 33)
(0, 33)
(8, 30)
(0, 33)
(8, 30)
(13, 30)
(0, 33)
(8, 30)
(5, 30)
(0, 33)
(0, 33)
(0, 33)
(1, 33)
(7, 30)
(0, 33)
(1, 33)
(6, 30)
(0, 33)
(0, 33)
(10, 30)
(6, 30)
(0, 33)
(7, 30)
(0, 33)
(8, 30)
(7, 30)
(8, 30)
(7, 30)
(8, 30)
(10, 30)
(0, 33)
(0, 33)
(0, 33)
(6, 30)
(0, 33)
(6, 30)
(0, 33)
(0, 33)
(0, 33)
(12, 30)
(10, 30)
(0, 33)
(9, 30)
(8, 30)
(0, 33)
(7, 30)
(6, 30)
(0, 33)
(7, 30)
(11, 30)
(0, 33)
(11, 30)
(7, 30)
(6, 30)
(10, 30)
(7, 30)
(0, 33)
(0, 33)
(1, 33)
(0, 33)
(0, 33)
(8, 30)
(0, 33)
(6, 30)
(8, 30)
(6, 30)
(4, 30)
(7, 30)
(11, 30)
(8, 30)
(0, 33)
(0, 33)
(7, 30)
(1, 33)
(0, 33)
(7, 

In [20]:
for key, value in data_ref.items():
    print(f"{key}: Length is {len(value)}")

chr7_83373052_83373141_83373122_83373127_TTCAG_T: Length is 432
chr1_150153438_150153527_150153491_150153492_A_AT: Length is 432
chr9_91159528_91159617_91159571_91159572_G_GAGGA: Length is 432
chr10_106140219_106140308_106140222_106140224_GA_G: Length is 432
chr10_79968168_79968257_79968231_79968232_G_GT: Length is 432
chr2_229923195_229923284_229923252_229923256_CAAT_C: Length is 432
chr12_7620216_7620305_7620270_7620275_ATAAT_A: Length is 432
chr1_89589498_89589587_89589557_89589558_C_CT: Length is 432
chrX_13722196_13722285_13722207_13722210_CAA_C: Length is 432
chr19_23897276_23897365_23897307_23897308_A_ATTTT: Length is 432
chr19_23897276_23897365_23897307_23897309_AT_A: Length is 432
chr17_36001577_36001666_36001606_36001607_C_CAGGGCAG: Length is 432
chrX_13722196_13722285_13722210_13722211_A_C: Length is 432
chr7_121686319_121686408_121686331_121686332_C_CAGAG: Length is 432
chr11_62855768_62855857_62855818_62855819_C_CA: Length is 432
chr19_23897276_23897365_23897307_23897315_A

In [21]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

ValueError: All arrays must be of the same length

In [ ]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

In [ ]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

In [ ]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

In [ ]:
result_df_ref.to_csv(output_dir+"df_ref_score.tsv", sep="\t", index=False)
result_df_alt.to_csv(output_dir+"df_alt_score.tsv", sep="\t", index=False)
result_df_log_odd.to_csv(output_dir+"df_log_odd_score.tsv", sep="\t", index=False)
result_df_score_change.to_csv(output_dir+"df_score_change.tsv", sep="\t", index=False)